In [1]:
import tweepy
import dotenv
import pandas as pd

dotenv.load_dotenv()

# Authentication credentials
consumer_key = dotenv.get_key(dotenv.find_dotenv(), 'CONSUMER_KEY')
consumer_secret = dotenv.get_key(dotenv.find_dotenv(), 'CONSUMER_SECRET')
access_token = dotenv.get_key(dotenv.find_dotenv(), 'ACCESS_TOKEN')
access_secret = dotenv.get_key(dotenv.find_dotenv(), 'ACCESS_SECRET')

# Authenticate
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

# Collect tweets based on keyword
query = 'python'
tweets = tweepy.Cursor(api.search_tweets, q=query, lang='en', tweet_mode='extended').items(100)

# Store tweets as text
tweet_texts = [tweet.full_text for tweet in tweets]

# Load into DataFrame for analysis
df = pd.DataFrame(tweet_texts, columns=['tweet'])

# Example: basic statistics
print(df['tweet'].str.len().describe())


Forbidden: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product

In [ ]:
# If you store 'created_at':
tweets_data = [{'text': tweet.full_text, 'created_at': tweet.created_at} for tweet in tweets]
df = pd.DataFrame(tweets_data)

# Aggregating number of tweets per day
df['date'] = pd.to_datetime(df['created_at']).dt.date
daily_counts = df.groupby('date').size()
daily_counts.plot(kind='bar', figsize=(10, 5), title='Number of Tweets per Day')
